# Brochure Builder – Full Business Solution

## Project Overview
This project extends the Day 1 prototype into a complete business-ready solution.  
Given a company name and its primary website, the system automatically generates a polished brochure suitable for prospective clients, investors, and potential recruits.

## Business Challenge
Design and implement a product that:
- Ingests a company’s main website.
- Extracts and organizes key information about the business.
- Produces a clear, compelling brochure that non-technical stakeholders can use.

## Real-World Use
This solution can be adapted for:
- Sales and marketing collateral.
- Investor pitch materials.
- Employer branding and recruiting assets.


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper_new import fetch_website_links_dynamic, fetch_website_contents_dynamic
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'  # Fixed: was 'gpt-5-nano'
openai = OpenAI()


API key looks good so far


In [3]:
links = fetch_website_links_dynamic("https://spearmint.online")
content = fetch_website_contents_dynamic("https://spearmint.online")


In [5]:
links


['tel:+9187560 80645',
 'mailto:info@spearmint.online',
 'tel:+9187560 80645',
 'mailto:info@spearmint.online',
 'https://spearmint.online/',
 'https://spearmint.online/',
 'https://spearmint.online',
 'https://spearmint.online/web-design/',
 'https://spearmint.online/reactjs-development/',
 'https://spearmint.online/next-js-development/',
 'https://spearmint.online/node-js-development/',
 'https://spearmint.online/angular-js-development/',
 'https://spearmint.online/python-development/',
 'https://spearmint.online',
 'https://spearmint.online/cross-platform/',
 'https://spearmint.online/app-designing/',
 'https://spearmint.online/native-apps/',
 'https://spearmint.online/wearables-app/',
 'https://spearmint.online/progressive-web-app/',
 'https://spearmint.online/ott-apps/',
 'https://spearmint.online',
 'https://spearmint.online/smart-contracts-development/',
 'https://spearmint.online/smart-contracts-audit/',
 'https://spearmint.online/web3-development-services/',
 'https://spearmin

In [7]:
content

'Blockchain Development Services | Spearmint\n\n+9187560 80645\ninfo@spearmint.online\n+9187560 80645\ninfo@spearmint.online\nHOME\nWeb\nWeb Designing\nReact JS Development\nNext JS Development\nNode JS Development\nAngular JS Development\nPython Development\nMobile\nCross Platform\nApp Designing\nNative Apps\nWearable Apps\nProgressive Web App\nOTT Apps\nBlockchain\nSmart Contract Development\nSmart Contract Audit\nWeb3 Development\nToken Development\nToken Development Company\nBlockchain Social Media Development\nServices\nWeb Application\nData Science\nMachine Learning Development\nIoT Development\nDevOps Services\nSaaS Development Maintenance\nDigital Marketing\nWhitepaper Writing\nSOCIAL MEDIA MARKETING (SMM)\nSOCIAL MEDIA OPTIMIZATION (SMO)\nPAY PER CLICK (PPC)\nSearch Engine Optimization (SEO)\nInfluencer Marketing\nGrowth & Performance Marketing\nVoice & AI Marketing\nVideo & Youtube Marketing\nInfluencer & Affiliate Marketing\nLead Generation\nConversion Rate Optimization\nMar

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [8]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
def get_links_user_prompt(url):
    """Generate prompt with links from spearmint.online"""
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, tel: or mailto: links.

Links (some might be relative links):

"""
    # Use your dynamic scraper instead of fetch_website_links
    links = fetch_website_links_dynamic(url)
    user_prompt += "\n".join(links)
    return user_prompt


In [10]:
print(get_links_user_prompt("https://spearmint.online"))



Here is the list of links on the website https://spearmint.online -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, tel: or mailto: links.

Links (some might be relative links):

tel:+9187560 80645
mailto:info@spearmint.online
tel:+9187560 80645
mailto:info@spearmint.online
https://spearmint.online/
https://spearmint.online/
https://spearmint.online
https://spearmint.online/web-design/
https://spearmint.online/reactjs-development/
https://spearmint.online/next-js-development/
https://spearmint.online/node-js-development/
https://spearmint.online/angular-js-development/
https://spearmint.online/python-development/
https://spearmint.online
https://spearmint.online/cross-platform/
https://spearmint.online/app-designing/
https://spearmint.online/native-apps/
https://spearmint.online/wearables-app/
https://spearmint.online/progressive-web-app/
https:

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links


In [12]:
# Get filtered, relevant links for brochure
relevant_links = select_relevant_links("https://spearmint.online")
print(json.dumps(relevant_links, indent=2))


Selecting relevant links for https://spearmint.online by calling gpt-5-nano
Found 5 relevant links
{
  "links": [
    {
      "type": "about page",
      "url": "https://spearmint.online/about-us/"
    },
    {
      "type": "careers page",
      "url": "https://spearmint.online/careers/"
    },
    {
      "type": "services page",
      "url": "https://spearmint.online/services/"
    },
    {
      "type": "contact page",
      "url": "https://spearmint.online/contact-us/"
    },
    {
      "type": "brochure pdf",
      "url": "https://spearmint.online/wp-content/uploads/2025/11/Spearmint.pdf"
    }
  ]
}


In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn/resource page', 'url': 'https://huggingface.co/learn'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [ ]:
# Step 1: Fetch Page and Relevant Links (with rate limiting)

import time

def fetch_page_and_all_relevant_links(url, max_links=3):
    """Fetch main page and limited relevant links with rate limiting"""
    print(f"Fetching main page: {url}")
    contents = fetch_website_contents_dynamic(url)
    
    print(f"Selecting relevant links...")
    relevant_links = select_relevant_links(url)
    
    # Limit to first N links to avoid browser crashes
    links_to_fetch = relevant_links['links'][:max_links]
    print(f"Fetching {len(links_to_fetch)} most relevant pages...")
    
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    
    for i, link in enumerate(links_to_fetch):
        print(f"Fetching {i+1}/{len(links_to_fetch)}: {link['type']}")
        try:
            result += f"\n\n### Link: {link['type']}\n"
            result += fetch_website_contents_dynamic(link["url"])
            time.sleep(2)  # 2 second delay between requests
        except Exception as e:
            print(f"Error fetching {link['url']}: {e}")
            result += f"[Could not fetch this page]\n"
    
    return result


In [15]:
# Step 2: Test the Fetcher
print(fetch_page_and_all_relevant_links("https://spearmint.online"))


Selecting relevant links for https://spearmint.online by calling gpt-5-nano
Found 3 relevant links
## Landing Page:

Blockchain Development Services | Spearmint

+9187560 80645
info@spearmint.online
+9187560 80645
info@spearmint.online
HOME
Web
Web Designing
React JS Development
Next JS Development
Node JS Development
Angular JS Development
Python Development
Mobile
Cross Platform
App Designing
Native Apps
Wearable Apps
Progressive Web App
OTT Apps
Blockchain
Smart Contract Development
Smart Contract Audit
Web3 Development
Token Development
Token Development Company
Blockchain Social Media Development
Services
Web Application
Data Science
Machine Learning Development
IoT Development
DevOps Services
SaaS Development Maintenance
Digital Marketing
Whitepaper Writing
SOCIAL MEDIA MARKETING (SMM)
SOCIAL MEDIA OPTIMIZATION (SMO)
PAY PER CLICK (PPC)
Search Engine Optimization (SEO)
Influencer Marketing
Growth & Performance Marketing
Voice & AI Marketing
Video & Youtube Marketing
Influencer & 

In [16]:
# Step 3: Define Brochure Prompts
# Professional version
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Optional: Humorous version
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
# Step 4: Generate User Prompt
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt


In [19]:
# Step 5: Create Brochure (Non-Streaming)
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,  # Uses your gpt-4o-mini
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Generate brochure
create_brochure("Spearmint", "https://spearmint.online")

Selecting relevant links for https://spearmint.online by calling gpt-5-nano
Found 5 relevant links


WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=143.0.7499.40)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7617a8255
	0x7ff7617a82b0
	0x7ff76158165d
	0x7ff76157e1dd
	0x7ff76156eb69
	0x7ff761570974
	0x7ff76156f10f
	0x7ff76156e871
	0x7ff76156e5bd
	0x7ff76156c20a
	0x7ff76156c9f2
	0x7ff761585950
	0x7ff76162bad4
	0x7ff761601fda
	0x7ff76162ac97
	0x7ff7615cac29
	0x7ff7615cba93
	0x7ff761abfff0
	0x7ff761aba930
	0x7ff761ad9096
	0x7ff7617c5754
	0x7ff7617ce6fc
	0x7ff7617b1974
	0x7ff7617b1b25
	0x7ff761797852
	0x7fff185b7374
	0x7fff19ebcc91


In [ ]:
# Step 6: Streaming Version (Typewriter Effect)
def stream_brochure(company_name, url):
    """Stream brochure with typewriter animation"""
    stream = openai.chat.completions.create(
        model=MODEL,  # Uses your gpt-4o-mini
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

# Generate streaming brochure
stream_brochure("Spearmint", "https://spearmint.online")
